# Get-started (Local)
This guide will walk you through the main user journey of prompt flow code-first experience. You will learn how to create and develop your first prompt flow, test and evaluate it, then deploy it to production.

## Step 1: Create and develop a new bulk run

In [1]:
import promptflow as pf
from promptflow.sdk.entities import AzureOpenAIConnection, BulkInputs, EvalInputs
from promptflow.sdk._pf_client import PFClient

# client can help manage your runs and connections.
client = PFClient()

### Load flow and set bulk inputs

In [2]:

flow_path = "../../flows/standard/web-classification"
bulk_inputs = BulkInputs(
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    inputs_mapping={"url": "${data.url}"},  # map the url field from the data to the url input of the flow
)

### Add connection
Since flow `web-classification` uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

In [3]:
connection = AzureOpenAIConnection(
    name="test_connection_1",
    type="AzureOpenAI",
    api_key="<test_key>",
    api_base="<test_base>",
    api_type="azure",
    api_version="<test_version>",
)

client.connections.create_or_update(connection)

### Run bulk

In [4]:
# use the first variant of the summarize_text_content node.
# if "node_variant" is not specified in "run_bulk" function, the default variant will be used.
node_variant="${summarize_text_content.variant_1}"

run = pf.run_bulk(
    flow=flow_path,
    inputs=bulk_inputs,
    node_variant=node_variant,  
)

bulk_run_name = run.name
# pf.stream(run)

2023-07-11 10:55:59 +0800   32552 execution.flow     INFO     Root flow run not found in run storage 'DummyRunStorage', will create a new root flow run. Run id: '31f827dc-a74c-4df7-bc43-fca73288d6d1', flow id: 'web_classification'
2023-07-11 10:55:59 +0800   32552 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: 31f827dc-a74c-4df7-bc43-fca73288d6d1_fetch_text_content_from_url_0
2023-07-11 10:55:59 +0800   32552 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: 31f827dc-a74c-4df7-bc43-fca73288d6d1_fetch_text_content_from_url_1
2023-07-11 10:55:59 +0800   32552 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: 31f827dc-a74c-4df7-bc43-fca73288d6d1_fetch_text_content_from_url_2
2023-07-11 10:56:01 +0800   32552 execution.flow     INFO     Node fetch_text_content_from_url completes.
2023-07-11 10:56:01 +0800   32552 execution.flow     INFO     Executing node summarize_text_content. node

In [ ]:
pf.show_details([run])

In [ ]:
pf.visualize([run])

## Step 2: Evaluate your bulk flow run

In [6]:
eval_flow_path = "../../flows/standard/classification_accuracy_evaluation"
eval_inputs = EvalInputs(
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    inputs_mapping={"groundtruth": "${data.answer}","prediction": "${variant.outputs.category}"},  # map the url field from the data to the url input of the flow
    variant=bulk_run_name,  # use the bulk run name as the variant
)

eval_run = pf.run_eval(
    flow=eval_flow_path,
    inputs=eval_inputs,
)

2023-07-11 10:58:21 +0800   32552 execution.flow     INFO     Root flow run not found in run storage 'DummyRunStorage', will create a new root flow run. Run id: 'e0642dd5-8651-4602-a1e8-3defa9fd8362', flow id: 'default_flow_id'
2023-07-11 10:58:21 +0800   32552 execution.flow     INFO     Executing node grade. node run id: e0642dd5-8651-4602-a1e8-3defa9fd8362_grade_0
2023-07-11 10:58:21 +0800   32552 execution.flow     INFO     Executing node grade. node run id: e0642dd5-8651-4602-a1e8-3defa9fd8362_grade_1
2023-07-11 10:58:21 +0800   32552 execution.flow     INFO     Executing node grade. node run id: e0642dd5-8651-4602-a1e8-3defa9fd8362_grade_2
2023-07-11 10:58:21 +0800   32552 execution.flow     INFO     Node grade completes.
2023-07-11 10:58:21 +0800   32552 execution.bulk     INFO     Finished 1 / 3 lines.
2023-07-11 10:58:21 +0800   32552 execution.flow     INFO     Node grade completes.
2023-07-11 10:58:21 +0800   32552 execution.bulk     INFO     Finished 2 / 3 lines.
2023-07-11

Writing output to D:\programs\prompt-flow\flows\standard\classification_accuracy_evaluation\.runs\b4950724-ca52-4478-9e02-ba1afebc60bb\details.jsonl...


In [ ]:
pf.show_details(eval_run)

In [ ]:
pf.show_metrics(eval_run)

## Step 3: Deploy the flow (WIP)